In [2]:
import pandas as pd
import numpy as np
import re
import pickle
import gensim

from wordcloud import WordCloud
from gensim import corpora
from gensim import models
from gensim.corpora.dictionary import Dictionary

from collections import defaultdict
from gensim.parsing.preprocessing import STOPWORDS

In [27]:
import string
import logging

In [28]:
df1 = pd.read_pickle('dfs/FP4')
df4 = df1[['title','objective']]
df4 = df4.dropna(how='any')
df4['merged'] = df4['title'] + ' ' + df4['objective']

In [29]:
df1 = pd.read_pickle('dfs/FP5')
df5= df1[['title','objective']]
df5 = df5.dropna(how='any')
df5['merged'] = df5['title'] + ' ' + df5['objective']

In [30]:
df1 = pd.read_pickle('dfs/FP6')
df6= df1[['title','objective']]
df6 = df6.dropna(how='any')
df6['merged'] = df6['title'] + ' ' + df6['objective']

In [31]:
df1 = pd.read_pickle('dfs/FP7')
df7= df1[['title','objective']]
df7 = df7.dropna(how='any')
df7['merged'] = df7['title'] + ' ' + df7['objective']

In [33]:
df1 = pd.read_pickle('dfs/H2020')
df20= df1[['title','objective']]
df20 = df20.dropna(how='any')
df20['merged'] = df20['title'] + ' ' + df20['objective']

In [34]:
# objectives = df1['merged']
objectives = pd.concat([ df['objective'] for df in [df4, df5, df6, df7, df20] ])

In [35]:
RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])

objectives = objectives.str.lower().str.replace('%l', '').str.replace(RE_PUNCTUATION, ' ')

In [36]:
objectives_split = objectives.str.strip().str.split()
objectives_split = objectives_split.apply(lambda tokens: [token for token in tokens if len(token) > 2])
objectives_split = objectives_split.apply(lambda tokens: [token for token in tokens if not(token.isdigit())])
objectives_split.head(2)

0                                                   []
1    [understand, the, mechanisms, leading, the, fo...
Name: objective, dtype: object

In [37]:
list_stopwords = ['data','will', 'develop', 'development', 'project', 'research', 'new', 'use', 'europe', 'european', 'based']
additional_stopwords = set(list_stopwords)
stopwords = set(STOPWORDS) | additional_stopwords

objectives_split = objectives_split.apply(lambda tokens: [token for token in tokens if token not in stopwords])
objectives_split.head(2)

0                                                   []
1    [understand, mechanisms, leading, formation, n...
Name: objective, dtype: object

In [38]:
frequency = defaultdict(int)
for text in objectives_split:
    for token in text:
        frequency[token] += 1

objectives_split = objectives_split.apply(lambda tokens: [token for token in tokens if (frequency[token] > 5)])

objectives_dictionary = Dictionary(objectives_split)

In [39]:
class ObjectivesCorpus(corpora.textcorpus.TextCorpus):
    def get_texts(self):
        return iter(self.input)
    def __calc_corpus_size__(self):
        logging.info('Calculating corpus size')
        self.length = 0
        self.num_words = 0
        for doc in self.get_texts():
            self.length += 1
            self.num_words += len(doc)
    def __len__(self):
        """Define this so we can use `len(corpus)`"""
        if 'length' not in self.__dict__:
            self.__calc_corpus_size__()
        return self.length
    def __str__(self):
        if 'num_words' not in self.__dict__:
            self.__calc_corpus_size__()
        return (str(self.length) + ' documents, ' + str(self.num_words)
                + ' words')
            

objectives_corpus = ObjectivesCorpus(objectives_split)

In [40]:
lda = gensim.models.ldamodel.LdaModel(corpus = objectives_corpus, 
                                        id2word = objectives_dictionary, 
                                        num_topics = 10,
                                        passes=2,
                                        random_state = np.random.seed(12),
                                        iterations = 500)

In [41]:
lda.show_topics()

[(0,
  u'0.013*"systems" + 0.008*"theory" + 0.008*"time" + 0.007*"design" + 0.006*"information" + 0.006*"tools" + 0.006*"applications" + 0.006*"models" + 0.006*"real" + 0.006*"analysis"'),
 (1,
  u'0.011*"clinical" + 0.011*"cancer" + 0.010*"health" + 0.009*"patients" + 0.008*"disease" + 0.008*"treatment" + 0.007*"molecular" + 0.006*"human" + 0.006*"drug" + 0.005*"care"'),
 (2,
  u'0.015*"materials" + 0.015*"quantum" + 0.013*"high" + 0.010*"properties" + 0.008*"light" + 0.008*"optical" + 0.008*"applications" + 0.008*"novel" + 0.008*"devices" + 0.006*"single"'),
 (3,
  u'0.009*"climate" + 0.008*"molecular" + 0.007*"understanding" + 0.007*"processes" + 0.006*"change" + 0.006*"models" + 0.006*"model" + 0.006*"chemistry" + 0.006*"time" + 0.006*"dynamics"'),
 (4,
  u'0.011*"innovation" + 0.010*"network" + 0.010*"services" + 0.010*"training" + 0.009*"support" + 0.009*"researchers" + 0.007*"activities" + 0.007*"business" + 0.006*"management" + 0.006*"information"'),
 (5,
  u'0.019*"market" + 0

In [24]:
from __future__ import division
import graphlab as gl
import pyLDAvis
import pyLDAvis.graphlab

In [25]:
pyLDAvis.enable_notebook()

In [42]:
import pyLDAvis.gensim

pyLDAvis.gensim.prepare(lda, objectives_corpus, objectives_dictionary)

/usr/local/lib/python2.7/dist-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      13.820449        1       1  0.178962 -0.169566
9      13.744508        1       2  0.044036 -0.192528
0      11.928951        1       3  0.068235  0.021437
7      10.058213        1       4 -0.218671 -0.081903
6       9.775038        1       5  0.145827  0.134760
5       8.671871        1       6  0.228891  0.033788
1       8.269963        1       7 -0.117586 -0.118200
2       8.154369        1       8  0.003340  0.241782
3       7.864801        1       9 -0.139687  0.048046
8       7.711837        1      10 -0.193348  0.082383, topic_info=      Category          Freq              Term         Total  loglift  logprob
term                                                                          
25698  Default  22795.000000            energy  22795.000000  30.0000  30.0000
33545  Default  16427.000000              cell  16427.000000  29.0000  29.0000
32364  Default  21233.000000            market  21233.000000  28.0000  28.0000
33058  Default   9717.000000           quantum   9717.000000  27.0000  27.0000
34062  Default  14505.000000             cells  14505.000000  26.0000  26.0000
24789  Default  12865.000000         materials  12865.000000  25.0000  25.0000
6251   Default  10966.000000          services  10966.000000  24.0000  24.0000
2529   Default  13278.000000        production  13278.000000  23.0000  23.0000
28492  Default  16903.000000        innovation  16903.000000  22.0000  22.0000
30481  Default  12695.000000            social  12695.000000  21.0000  21.0000
3016   Default  12432.000000              cost  12432.000000  20.0000  20.0000
32199  Default   7639.000000          clinical   7639.000000  19.0000  19.0000
5529   Default   9550.000000            policy   9550.000000  18.0000  18.0000
8242   Default  10328.000000       researchers  10328.000000  17.0000  17.0000
1008   Default  23373.000000        technology  23373.000000  16.0000  16.0000
11635  Default  26840.000000              high  26840.000000  15.0000  15.0000
15578  Default   8172.000000            cancer   8172.000000  14.0000  14.0000
30018  Default  12096.000000          training  12096.000000  13.0000  13.0000
1824   Default  11612.000000          business  11612.000000  12.0000  12.0000
31837  Default  12958.000000           network  12958.000000  11.0000  11.0000
18517  Default   8936.000000            health   8936.000000  10.0000  10.0000
17882  Default   8913.000000            theory   8913.000000   9.0000   9.0000
32178  Default   9683.000000        properties   9683.000000   8.0000   8.0000
3738   Default  25495.000000           systems  25495.000000   7.0000   7.0000
19983  Default  13487.000000           support  13487.000000   6.0000   6.0000
30657  Default  10825.000000         molecular  10825.000000   5.0000   5.0000
19159  Default   6113.000000          patients   6113.000000   4.0000   4.0000
16822  Default   6891.000000           climate   6891.000000   3.0000   3.0000
10426  Default  14093.000000      applications  14093.000000   2.0000   2.0000
6866   Default   7746.000000           disease   7746.000000   1.0000   1.0000
...        ...           ...               ...           ...      ...      ...
24289  Topic10   1362.582815           tissues   1424.874606   2.5177  -6.1370
14603  Topic10   1771.372438        activation   1927.707145   2.4778  -5.8746
3877   Topic10   3923.402201           physics   5391.253369   2.2446  -5.0794
3126   Topic10   3939.810858             brain   5508.491818   2.2273  -5.0753
13115  Topic10   1869.945464  characterization   2255.190274   2.3751  -5.8205
7617   Topic10   1724.535199         numerical   2158.238448   2.3381  -5.9014
15683  Topic10   1538.106689          neuronal   1917.597926   2.3419  -6.0158
37840  Topic10   1669.437366         synthetic   2202.014180   2.2855  -5.9339
9236   Topic10   2291.045425           organic   3754.522468   2.0685 